In [1]:
import pandas as pd

In [2]:
import json

In [3]:
with open('28-09_22-11_alldata.json','r') as j_in:
    data = json.load(j_in)

In [4]:
def json_unwrap(data: dict):
    """
    accepts a dict json gathered from asystom dashboard
    and outputs a list of pd.DataFrame tables 
    """
    
    names = [data['results'][n]['series'][0]['name'] for n in range(4)]
    cols = [data['results'][n]['series'][0]['columns'] for n in range(4)]
    dfs = [pd.DataFrame(data['results'][i]['series'][0]['values'],columns=cols[i]) for i in range(4)]
    for i in range(4): dfs[i].name = names[i]
    
    return dfs

In [31]:
def generate_cd_df(df:pd.DataFrame,n_clusters=4):
    '''
    accepts a pd.DataFrame obj
    returns a wider df with the distance from each column to its respective centroid column
    
    n_clusters (default 4) is the number of groups to cluster data into
    
    added columns begin with "cd_"
    '''
    
    import warnings
    warnings.filterwarnings("ignore")
    
    
    df_0 = dfs[0].copy()
    
    #seize down df with only numeric columns
    numeric_df_0 = df_0.select_dtypes('number')
    
    #remove useless timestamp
    numeric_df_0.drop('time',axis=1,inplace=True)
    
    #kmeans with 4 
    from sklearn.cluster import KMeans
    kmeans_0 = KMeans(init='k-means++',n_clusters=n_clusters)
    fitted_kmeans_df_0 = kmeans_0.fit(numeric_df_0)
        
    #retrieve the respective group label for each centroid
    centroids_labels = [fitted_kmeans_df_0.predict(pd.DataFrame(centroid).T)[0] for centroid in fitted_kmeans_df_0.cluster_centers_]
    
    #create a dict with centroid labels as keys and actual centroids as values
    centroids_with_labels = dict(zip(centroids_labels,fitted_kmeans_df_0.cluster_centers_))
    
    #create a list (then converted in df) with length matching the original df with centroids aligned with its respective data points
    #this allows to create a df to subtract and calculate distances 
    centroids_list = [centroids_with_labels[label] for label in fitted_kmeans_df_0.labels_]
    centroids_df_0 = pd.DataFrame(centroids_list)
    centroids_df_0.columns = numeric_df_0.columns
    
    #calc ditances between original df and centroids df
    cd_df_0 = numeric_df_0.subtract(centroids_df_0)
    cd_df_0.columns = ['cd_'+cn for cn in numeric_df_0.columns]
    
    wide_df = pd.concat([df_0,cd_df_0],axis=1)
    wide_df['kmeans_label'] = kmeans_0.labels_
    
    return wide_df
    

In [30]:
#usage

dfs = json_unwrap(data)

generate_cd_df(dfs[0],6)

,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,cd_vib_z_acc,cd_vib_z_f1,cd_vib_z_f2,cd_vib_z_f3,cd_vib_z_kurt,cd_vib_z_peak,cd_vib_z_root,cd_vib_z_vel,cd_vibra_custom,kmeans_label
0,1632822027177,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,258,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.389041,-2.791706e+00,-3.933983e+00,-1.071534e+00,0.548377,0.452701,-8.293181e-01,-0.713211,0.0,0
1,1632822331552,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.603113,0.428016,1.520943,0.162509,0.070954,...,-0.286238,-2.791706e+00,-3.933983e+00,-1.071534e+00,0.478643,0.444889,-8.293181e-01,-0.560433,0.0,0
2,1632822626555,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,1.068894,0.929274,1.863890,1.574731,0.604257,...,-0.299756,-2.791706e+00,-3.933983e+00,-1.071534e+00,0.981609,1.056337,-8.293181e-01,-0.609611,0.0,0
3,1632822926402,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.295262,-0.921836,-0.814832,0.993935,0.504692,...,-0.375708,-2.791706e+00,-3.933983e+00,-1.071534e+00,0.628949,0.529113,-8.293181e-01,-0.677427,0.0,0
4,1632823226427,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.441291,-0.576333,1.248798,2.314946,0.782788,...,-0.322608,-2.791706e+00,-3.933983e+00,-1.071534e+00,1.025487,0.682661,-8.293181e-01,-0.674551,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24387,1637580817598,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,513,0.117213,-0.862416,0.121430,-0.251412,0.460662,...,-0.178575,1.765255e-14,7.949197e-14,3.552714e-15,0.057619,-0.133733,5.773160e-15,-0.731915,0.0,1
24388,1637580937586,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,513,0.298033,-1.253810,-0.661358,-0.743516,0.405729,...,-0.253972,1.765255e-14,7.949197e-14,3.552714e-15,0.049037,-0.276949,5.773160e-15,-0.960831,0.0,1
24389,1637581057578,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,513,0.265989,-0.500777,-0.182988,-0.475720,0.023491,...,-0.216274,1.765255e-14,7.949197e-14,3.552714e-15,0.051561,-0.105089,5.773160e-15,-0.701393,0.0,1
24390,1637581182491,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,513,0.588717,-0.388623,0.677621,-0.342967,0.291287,...,-0.253972,1.765255e-14,7.949197e-14,3.552714e-15,-0.008011,-0.257853,5.773160e-15,-0.564044,0.0,1
